# TABLAS RIPS 

## Carga de los datos

Este jupyter notebook esta generando las tablas sobre un año específico de la fuente de datos RIPS

Para cada año se debe editar la ruta para la lectura del archivo inicial y las 9 rutas correspondientes a la ruta de cada tablet al crear los archivos de tipo parquet

Modificar el nombre de spark y la carpeta a cargar

In [166]:
from pyspark.sql import SparkSession

# Crear una sesión de Spark
spark = SparkSession.builder.appName("GeneraciónTablasRips_Todoslosdatos_2009").getOrCreate()

### Importante Definir la ruta del año para generar los datos, se realizó año por año por la cantidad de registros

In [167]:
# Leer los archivos Parquet de un año 
df1_parquet = spark.read.parquet("/rawdata/rips/rips_parquet_2009")

df=df1_parquet

In [168]:
# Contar el número de registros en el DataFrame unido
total_registros = df.count()

# Listar las columnas del DataFrame unido
columnas = df.columns

# Mostrar resultados
print(f"Total de registros en el DataFrame unido: {total_registros}")
print(f"Columnas del DataFrame unido: {columnas}")

Total de registros en el DataFrame unido: 154880279
Columnas del DataFrame unido: ['personaid', 'tipoeventoripsdesc', 'codigo', 'dxprincipal', 'dxegreso', 'finalidadprocedimientoscd', 'finalidadconsultacd', 'tipousuariocd', 'causaexternacd', 'prestador', 'estadosalidacd', 'tipodiagnosticoprincipalcd', 'codigoprocedimiento', 'municipiocd', 'fechaid', 'edad', 'sexodesc', 'costoconsulta', 'costoprocedimiento', 'numerodiasestancia', '__null_dask_index__']


In [169]:
# Contar el número de registros - da satisfactoriamente la cantidad de registros de toda la fuente RIPS 4’262.860.903 registros 
record_count = df.count()

# Mostrar el número de registros
#print(f"El número de registros en el archivo es: {record_count}")

In [170]:
# Contar el número total de columnas en el DataFrame ( no se tiene en cuenta neto a pagar)
total_columns = len(df.columns)

# Imprimir el número total de columnas
print("Número total de columnas en el DataFrame:", total_columns)

Número total de columnas en el DataFrame: 21


In [171]:
# Mostrar el esquema (columnas y tipos de datos)
df.printSchema()

# Listar las columnas
print(df.columns)

root
 |-- personaid: string (nullable = true)
 |-- tipoeventoripsdesc: string (nullable = true)
 |-- codigo: string (nullable = true)
 |-- dxprincipal: string (nullable = true)
 |-- dxegreso: string (nullable = true)
 |-- finalidadprocedimientoscd: string (nullable = true)
 |-- finalidadconsultacd: string (nullable = true)
 |-- tipousuariocd: string (nullable = true)
 |-- causaexternacd: string (nullable = true)
 |-- prestador: string (nullable = true)
 |-- estadosalidacd: string (nullable = true)
 |-- tipodiagnosticoprincipalcd: string (nullable = true)
 |-- codigoprocedimiento: string (nullable = true)
 |-- municipiocd: string (nullable = true)
 |-- fechaid: string (nullable = true)
 |-- edad: string (nullable = true)
 |-- sexodesc: string (nullable = true)
 |-- costoconsulta: string (nullable = true)
 |-- costoprocedimiento: string (nullable = true)
 |-- numerodiasestancia: string (nullable = true)
 |-- __null_dask_index__: long (nullable = true)

['personaid', 'tipoeventoripsdesc',

# Empieza el proceso de generación de los datos agrupados por atenciones

In [172]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window

# Crear una sesión de Spark
spark = SparkSession.builder.appName("DataFrame agrupado").getOrCreate()


# Cargar el archivo Parquet en un DataFrame
#df = df1

# Seleccionar solo las columnas deseadas (sin 'personaid_count' ya que no existe aún)
columns_to_keep = [
    'personaid', 'codigoprocedimiento', 'tipoeventoripsdesc', 'fechaid',
    'sexodesc', 'tipodiagnosticoprincipalcd', 'edad', 'dxegreso', 'dxprincipal', 'municipiocd'
]
df_filtered = df.select(*columns_to_keep)

# Agregar una nueva columna con el recuento distintivo de 'personaid'
window_spec = Window.partitionBy('personaid')
df_filtered = df_filtered.withColumn('personaid_count', F.count('personaid').over(window_spec))

# Mostrar las primeras filas del DataFrame filtrado para verificar
df_filtered.show()

# Mostrar los nombres de las columnas filtradas para verificar
#print("Nombres de las columnas filtradas:")
#print(df_filtered.columns)

# Agrupar por la columna 'personaid'
grouped_df = df_filtered.groupBy('personaid').agg(
    F.first('codigoprocedimiento').alias('codigoprocedimiento'),
    F.first('tipoeventoripsdesc').alias('tipoeventoripsdesc'),
    F.first('fechaid').alias('fechaid'),
    F.first('sexodesc').alias('sexodesc'),
    F.first('tipodiagnosticoprincipalcd').alias('tipodiagnosticoprincipalcd'),
    F.first('edad').alias('edad'),
    F.first('dxegreso').alias('dxegreso'),
    F.first('dxprincipal').alias('dxprincipal'),
    F.first('municipiocd').alias('municipiocd'),
    F.first('personaid_count').alias('personaid_count')
)

# Mostrar las primeras filas del DataFrame agrupado para verificar
grouped_df.show()


24/08/17 16:25:13 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


+---------+-------------------+--------------------+--------+---------+--------------------------+----+--------+-----------+-----------+---------------+
|personaid|codigoprocedimiento|  tipoeventoripsdesc| fechaid| sexodesc|tipodiagnosticoprincipalcd|edad|dxegreso|dxprincipal|municipiocd|personaid_count|
+---------+-------------------+--------------------+--------+---------+--------------------------+----+--------+-----------+-----------+---------------+
| 10000667|             937000|PROCEDIMIENTOS DE...|20091008|MASCULINO|                      null|  85|       0|       G219|      68001|             18|
| 10000667|             937000|PROCEDIMIENTOS DE...|20091005|MASCULINO|                      null|  85|       0|       G219|      68001|             18|
| 10000667|             937000|PROCEDIMIENTOS DE...|20091014|MASCULINO|                      null|  85|       0|       G219|      68001|             18|
| 10000667|             937000|PROCEDIMIENTOS DE...|20091006|MASCULINO|           

+---------+-------------------+--------------------+--------+---------+--------------------------+----+--------+-----------+-----------+---------------+
|personaid|codigoprocedimiento|  tipoeventoripsdesc| fechaid| sexodesc|tipodiagnosticoprincipalcd|edad|dxegreso|dxprincipal|municipiocd|personaid_count|
+---------+-------------------+--------------------+--------+---------+--------------------------+----+--------+-----------+-----------+---------------+
| 10000667|             937000|PROCEDIMIENTOS DE...|20091008|MASCULINO|                      null|  85|       0|       G219|      68001|             18|
| 10000670|                  1|   HOSPITALIZACIONES|20091207| FEMENINO|                      null|  58|    N390|       N390|      17380|             29|
| 10002095|             890301|           CONSULTAS|20090316| FEMENINO|                         1|  67|       0|       K589|      19001|             16|
| 10002373|             890201|           CONSULTAS|20090414|MASCULINO|           

In [173]:
# Contar el número de registros - me da satisfactoriamente la cantidad de regustros en cada año da 73'357.494 llego hasta stage 18
#llega a 30'434.249 con 2022
record_count = grouped_df.count()


# Mostrar el número de registros
print(f"El número de registros en el dataframe es: {record_count}")

El número de registros en el dataframe es: 17314834


Se generan un total de 73'357.494 registros (todos) para el año 2022 son 30'434.249  registros

In [174]:
 df_final =grouped_df


In [175]:
# Contar el número de registros 
record_count = df_final.count()

# Mostrar el número de registros
print(f"El número de registros en el archivo Parquet es: {record_count}")

El número de registros en el archivo Parquet es: 17314834


In [176]:
# Mostrar las primeras filas del DataFrame agrupado para verificar
df_final.show(19)

+---------+-------------------+--------------------+--------+---------+--------------------------+----+--------+-----------+-----------+---------------+
|personaid|codigoprocedimiento|  tipoeventoripsdesc| fechaid| sexodesc|tipodiagnosticoprincipalcd|edad|dxegreso|dxprincipal|municipiocd|personaid_count|
+---------+-------------------+--------------------+--------+---------+--------------------------+----+--------+-----------+-----------+---------------+
| 10000667|             937000|PROCEDIMIENTOS DE...|20091008|MASCULINO|                      null|  85|       0|       G219|      68001|             18|
| 10000670|             881332|PROCEDIMIENTOS DE...|20091204| FEMENINO|                      null|  58|       0|          1|      17380|             29|
| 10002095|             890201|           CONSULTAS|20090212| FEMENINO|                         1|  67|       0|       J040|      19001|             16|
| 10002373|             890201|           CONSULTAS|20090414|MASCULINO|           

In [177]:
#se escribe el parquet - No lo hago por que seria todos los datos por atenciones, mejor solo de cada tabla
#df_final.write.parquet("hdfs:/analyticsdata/todoRips.parquet") 

# Empieza el proceso de generación de tablas

In [178]:
# Cargar el archivo Parquet en un DataFrame
df2 = df_final

## Ejecutar hasta aca para tener los datos necesarios para generar las tablas (Run All Above) 

## 1. Agrupado por año y mes

In [378]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window

# Crear una sesión de Spark 
#spark = SparkSession.builder.appName("DataFrameTablas ").getOrCreate()


# Cargar el archivo Parquet en un DataFrame
#df2 = df_final




In [11]:
# Contar el número de registros - me da satisfactoriamente la cantidad de regustros en cada año 
record_count = df2.count()

# Mostrar el número de registros
print(f"El número de registros en el archivo Parquet es: {record_count}")

El número de registros en el archivo Parquet es: 29792722


In [380]:
# Mostrar el esquema (columnas y tipos de datos)
df2.printSchema()

# Listar las columnas
print(df2.columns)

root
 |-- personaid: string (nullable = true)
 |-- codigoprocedimiento: string (nullable = true)
 |-- tipoeventoripsdesc: string (nullable = true)
 |-- fechaid: string (nullable = true)
 |-- sexodesc: string (nullable = true)
 |-- tipodiagnosticoprincipalcd: string (nullable = true)
 |-- edad: string (nullable = true)
 |-- dxegreso: string (nullable = true)
 |-- dxprincipal: string (nullable = true)
 |-- municipiocd: string (nullable = true)
 |-- personaid_count: long (nullable = true)

['personaid', 'codigoprocedimiento', 'tipoeventoripsdesc', 'fechaid', 'sexodesc', 'tipodiagnosticoprincipalcd', 'edad', 'dxegreso', 'dxprincipal', 'municipiocd', 'personaid_count']


# resultado final agrupado año y mes, según el numero de personas y atenciones

In [381]:
# Seleccionar solo las columnas deseadas
columns_to_keep = [
    'personaid', 'codigoprocedimiento', 'tipoeventoripsdesc', 'fechaid',
    'sexodesc', 'tipodiagnosticoprincipalcd', 'edad', 'dxegreso', 'dxprincipal', 'municipiocd', 'personaid_count'
]
df_filtered = df2.select(*columns_to_keep)

# Convertir 'fechaid' a tipo fecha si es necesario
df_filtered = df_filtered.withColumn('fechaid', F.to_date('fechaid', 'yyyyMMdd'))

# Extraer año y mes de la columna 'fechaid'
df_filtered = df_filtered.withColumn('Año', F.year('fechaid'))
df_filtered = df_filtered.withColumn('Mes', F.month('fechaid'))

# Agrupar por año y mes
result_df = df_filtered.groupBy('Año', 'Mes').agg(
    F.countDistinct('personaid').alias('Número_de_Personas'),
    F.sum('personaid_count').alias('Número_de_Atenciones')
)

# Filtrar para mostrar los resultados de enero para comprobar
#result_january = result_df.filter((F.col('Año') == 2022) & (F.col('Mes') == 1))

# Mostrar los resultados para enero
#result_january.show()

# Mostrar las primeras filas del DataFrame completo para verificar
result_df.show()

# Imprimir los nombres de las columnas agrupadas para verificar
#print("Nombres de las columnas agrupadas:")
#print(result_df.columns)

+----+----+------------------+--------------------+
| Año| Mes|Número_de_Personas|Número_de_Atenciones|
+----+----+------------------+--------------------+
|2009|   4|           1401013|            12993231|
|2009|  10|           1301917|            11326896|
|2009|   9|           1537800|            14553472|
|2009|   6|           1366923|            12859201|
|2009|   3|           1581714|            13559072|
|2009|   1|           1538189|            12856988|
|2009|   2|           1451804|            12407530|
|2009|  12|           1377687|            11945323|
|2009|   8|           1398744|            13115232|
|2009|  11|           1345855|            11570363|
|2009|   7|           1592431|            14665408|
|2009|   5|           1420755|            13027561|
|null|null|                 2|                   2|
+----+----+------------------+--------------------+



In [382]:
result_df.show()

+----+----+------------------+--------------------+
| Año| Mes|Número_de_Personas|Número_de_Atenciones|
+----+----+------------------+--------------------+
|2009|   4|           1401527|            13037919|
|2009|  10|           1297923|            11379429|
|2009|   9|           1539961|            14600612|
|2009|   6|           1367806|            12616338|
|2009|   3|           1583948|            13611956|
|2009|   1|           1540094|            12874130|
|2009|   2|           1454861|            12478654|
|2009|  12|           1376048|            11807962|
|2009|   8|           1398431|            13080691|
|2009|  11|           1341906|            11513278|
|2009|   7|           1591560|            14738774|
|2009|   5|           1420767|            13140534|
|null|null|                 2|                   2|
+----+----+------------------+--------------------+



Modificar el nombre del archivo segun corresponda el año

### Importante modificar el nombre de los archivos antes de ejecutar segun el año que corresponda

In [383]:
#se escribe el parquet tabla 1
result_df.write.parquet("hdfs:/analyticsdata/1.Tabla_anio_mes_atenciones_2009.parquet") 

# 2. Agrupado por año, mes y sexo

In [384]:

# Seleccionar solo las columnas deseadas
columns_to_keep = [
    'personaid', 'codigoprocedimiento', 'tipoeventoripsdesc', 'fechaid',
    'sexodesc', 'tipodiagnosticoprincipalcd', 'edad', 'dxegreso', 'dxprincipal', 'municipiocd', 'personaid_count'
]
df_filtered = df2.select(*columns_to_keep)

# Convertir 'fechaid' a tipo fecha si es necesario
df_filtered = df_filtered.withColumn('fechaid', F.to_date('fechaid', 'yyyyMMdd'))

# Extraer año y mes de la columna 'fechaid'
df_filtered = df_filtered.withColumn('Año', F.year('fechaid'))
df_filtered = df_filtered.withColumn('Mes', F.month('fechaid'))

# Agrupar por año, mes y sexodesc
result_df = df_filtered.groupBy('Año', 'Mes', 'sexodesc').agg(
    F.countDistinct('personaid').alias('Número_de_Personas'),
    F.sum('personaid_count').alias('Número_de_Atenciones')
)

# Mostrar los resultados
result_df.show(100)

# Imprimir los nombres de las columnas agrupadas para verificar
print("Nombres de las columnas agrupadas:")
print(result_df.columns)


+----+----+-----------+------------------+--------------------+
| Año| Mes|   sexodesc|Número_de_Personas|Número_de_Atenciones|
+----+----+-----------+------------------+--------------------+
|2009|   8|   FEMENINO|            790112|             8341582|
|2009|   2|  MASCULINO|            604769|             4387755|
|2009|   7|   FEMENINO|            905826|             9347695|
|2009|   7|  MASCULINO|            673328|             5237076|
|2009|   4|       null|             11488|               80251|
|2009|   9|   FEMENINO|            869371|             9395415|
|2009|  10|       null|             10386|               69137|
|2009|   9|       null|             12716|               86888|
|2009|   3|   FEMENINO|            905253|             8678124|
|2009|  12|  MASCULINO|            564164|             4242733|
|2009|   6|       null|             11389|               89278|
|2009|   4|  MASCULINO|            586327|             4757885|
|2009|  10|  MASCULINO|            53699

In [385]:
#se escribe el parquet tabla 2
result_df.write.parquet("hdfs:/analyticsdata/2.Tabla_anio_mes_sexo_2009.parquet") 

# 3. Agrupado por año, mes y municipio

In [386]:

# Seleccionar solo las columnas deseadas
columns_to_keep = [
    'personaid', 'codigoprocedimiento', 'tipoeventoripsdesc', 'fechaid',
    'sexodesc', 'tipodiagnosticoprincipalcd', 'edad', 'dxegreso', 'dxprincipal', 'municipiocd', 'personaid_count'
]
df_filtered = df2.select(*columns_to_keep)

# Convertir 'fechaid' a tipo fecha si es necesario
df_filtered = df_filtered.withColumn('fechaid', F.to_date('fechaid', 'yyyyMMdd'))

# Extraer año y mes de la columna 'fechaid'
df_filtered = df_filtered.withColumn('Año', F.year('fechaid'))
df_filtered = df_filtered.withColumn('Mes', F.month('fechaid'))

# Agrupar por año, mes y municipioid
result_df = df_filtered.groupBy('Año', 'Mes', 'municipiocd').agg(
    F.countDistinct('personaid').alias('Número_de_Personas'),
    F.sum('personaid_count').alias('Número_de_Atenciones')
)

# Mostrar los resultados
result_df.show(100)

# Imprimir los nombres de las columnas agrupadas para verificar
print("Nombres de las columnas agrupadas:")
print(result_df.columns)


+----+---+-----------+------------------+--------------------+
| Año|Mes|municipiocd|Número_de_Personas|Número_de_Atenciones|
+----+---+-----------+------------------+--------------------+
|2009|  1|      41396|              2114|               10421|
|2009|  1|      23090|               324|                2950|
|2009|  9|      76020|               771|                5083|
|2009|  2|      20787|               351|                2859|
|2009| 11|      23090|              1288|                8116|
|2009|  9|      25506|               172|                 993|
|2009|  3|      52573|               380|                4424|
|2009|  3|      15740|               167|                 765|
|2009| 11|      25769|               338|                3240|
|2009|  1|      05607|               290|                1656|
|2009|  3|      25486|               305|                1971|
|2009|  6|      23686|               762|                2651|
|2009|  2|      25486|               206|              

In [387]:
#se escribe el parquet tabla 3
result_df.write.parquet("hdfs:/analyticsdata/3.Tabla_anio_mes_municipio_2009.parquet") 

# 4. Agrupado por año, mes y tipoeventoripsdesc

In [388]:
# Seleccionar solo las columnas deseadas
columns_to_keep = [
    'personaid', 'codigoprocedimiento', 'tipoeventoripsdesc', 'fechaid',
    'sexodesc', 'tipodiagnosticoprincipalcd', 'edad', 'dxegreso', 'dxprincipal', 'municipiocd', 'personaid_count'
]
df_filtered = df2.select(*columns_to_keep)

# Convertir 'fechaid' a tipo fecha si es necesario
df_filtered = df_filtered.withColumn('fechaid', F.to_date('fechaid', 'yyyyMMdd'))

# Extraer año y mes de la columna 'fechaid'
df_filtered = df_filtered.withColumn('Año', F.year('fechaid'))
df_filtered = df_filtered.withColumn('Mes', F.month('fechaid'))

# Agrupar por año, mes y tipoeventoripsdesc
result_df = df_filtered.groupBy('Año', 'Mes', 'tipoeventoripsdesc').agg(
    F.countDistinct('personaid').alias('Número_de_Personas'),
    F.sum('personaid_count').alias('Número_de_Atenciones')
)

# Mostrar los resultados
result_df.show(100)

# Imprimir los nombres de las columnas agrupadas para verificar
print("Nombres de las columnas agrupadas:")
print(result_df.columns)

+----+----+--------------------+------------------+--------------------+
| Año| Mes|  tipoeventoripsdesc|Número_de_Personas|Número_de_Atenciones|
+----+----+--------------------+------------------+--------------------+
|2009|   1|   HOSPITALIZACIONES|             21025|              270452|
|2009|   6|           CONSULTAS|            931099|             7700408|
|2009|   2|PROCEDIMIENTOS DE...|            408456|             4218743|
|2009|   8|PROCEDIMIENTOS DE...|            398817|             4629864|
|2009|  10|PROCEDIMIENTOS DE...|            442420|             4246999|
|2009|   4|           CONSULTAS|            977157|             8031547|
|2009|  11|PROCEDIMIENTOS DE...|            485866|             4436310|
|2009|   4|           URGENCIAS|             33484|              340359|
|2009|  12|           URGENCIAS|             38004|              385069|
|2009|   2|   HOSPITALIZACIONES|             16824|              261764|
|2009|   9|PROCEDIMIENTOS DE...|            431495|

In [389]:
#se escribe el parquet tabla 4
result_df.write.parquet("hdfs:/analyticsdata/4.Tabla_anio_mes_tipoeventoripsdesc_2009.parquet") 

# 5. Agrupado por año, mes y tipodiagnosticoprincipalcd

In [390]:
# Seleccionar solo las columnas deseadas
columns_to_keep = [
    'personaid', 'codigoprocedimiento', 'tipoeventoripsdesc', 'fechaid',
    'sexodesc', 'tipodiagnosticoprincipalcd', 'edad', 'dxegreso', 'dxprincipal', 'municipiocd', 'personaid_count'
]
df_filtered = df2.select(*columns_to_keep)

# Convertir 'fechaid' a tipo fecha si es necesario
df_filtered = df_filtered.withColumn('fechaid', F.to_date('fechaid', 'yyyyMMdd'))

# Extraer año y mes de la columna 'fechaid'
df_filtered = df_filtered.withColumn('Año', F.year('fechaid'))
df_filtered = df_filtered.withColumn('Mes', F.month('fechaid'))

# Agrupar por año, mes y tipodiagnosticoprincipalcd
result_df = df_filtered.groupBy('Año', 'Mes', 'tipodiagnosticoprincipalcd').agg(
    F.countDistinct('personaid').alias('Número_de_Personas'),
    F.sum('personaid_count').alias('Número_de_Atenciones')
)

# Mostrar los resultados
result_df.show(100)

# Imprimir los nombres de las columnas agrupadas para verificar
print("Nombres de las columnas agrupadas:")
print(result_df.columns)

+----+----+--------------------------+------------------+--------------------+
| Año| Mes|tipodiagnosticoprincipalcd|Número_de_Personas|Número_de_Atenciones|
+----+----+--------------------------+------------------+--------------------+
|2009|   6|                         2|            283054|             2135755|
|2009|   2|                         1|            524852|             3898398|
|2009|  12|                         1|            420070|             3108949|
|2009|   3|                         1|            556479|             4167619|
|2009|   4|                      null|            424501|             5131799|
|2009|  10|                      null|            516514|             5181543|
|2009|   9|                         1|            548132|             4515160|
|2009|   9|                      null|            506740|             5595877|
|2009|   1|                         2|            312842|             2161180|
|2009|   1|                         3|            18

In [391]:
#se escribe el parquet tabla 5
result_df.write.parquet("hdfs:/analyticsdata/5.Tabla_anio_mes_tipodiagnosticoprincipalcd_2009.parquet") 

# 6. Agrupado por año, mes y dxprincipal

In [392]:
# Seleccionar solo las columnas deseadas
columns_to_keep = [
    'personaid', 'codigoprocedimiento', 'tipoeventoripsdesc', 'fechaid',
    'sexodesc', 'tipodiagnosticoprincipalcd', 'edad', 'dxegreso', 'dxprincipal', 'municipiocd', 'personaid_count'
]
df_filtered = df2.select(*columns_to_keep)

# Convertir 'fechaid' a tipo fecha si es necesario
df_filtered = df_filtered.withColumn('fechaid', F.to_date('fechaid', 'yyyyMMdd'))

# Extraer año y mes de la columna 'fechaid'
df_filtered = df_filtered.withColumn('Año', F.year('fechaid'))
df_filtered = df_filtered.withColumn('Mes', F.month('fechaid'))

# Agrupar por año, mes y dxprincipal
result_df = df_filtered.groupBy('Año', 'Mes', 'dxprincipal').agg(
    F.countDistinct('personaid').alias('Número_de_Personas'),
    F.sum('personaid_count').alias('Número_de_Atenciones')
)

# Mostrar los resultados
result_df.show(100)

# Imprimir los nombres de las columnas agrupadas para verificar
print("Nombres de las columnas agrupadas:")
print(result_df.columns)

+----+---+-----------+------------------+--------------------+
| Año|Mes|dxprincipal|Número_de_Personas|Número_de_Atenciones|
+----+---+-----------+------------------+--------------------+
|2009|  1|       R42X|              6406|               55591|
|2009|  2|       D252|                57|                 629|
|2009|  7|       E782|              3873|               38509|
|2009|  3|       S025|              1209|                9173|
|2009|  6|       M791|              2376|               15153|
|2009|  4|       O800|              1825|               36747|
|2009|  3|       J219|              2641|               30203|
|2009| 11|       S923|               133|                 968|
|2009| 12|       Z018|              1358|               18789|
|2009|  2|       I279|                45|                 396|
|2009| 12|       S800|              1001|                5900|
|2009|  4|       B351|               541|                3666|
|2009|  9|       M708|                53|              

In [393]:
#se escribe el parquet tabla 6
result_df.write.parquet("hdfs:/analyticsdata/6.Tabla_anio_mes_dxprincipal_2009.parquet") 

# 7. Agrupado por año, mes y codigoprocedimiento

In [394]:
# Seleccionar solo las columnas deseadas
columns_to_keep = [
    'personaid', 'codigoprocedimiento', 'tipoeventoripsdesc', 'fechaid',
    'sexodesc', 'tipodiagnosticoprincipalcd', 'edad', 'dxegreso', 'dxprincipal', 'municipiocd', 'personaid_count'
]
df_filtered = df2.select(*columns_to_keep)

# Convertir 'fechaid' a tipo fecha si es necesario
df_filtered = df_filtered.withColumn('fechaid', F.to_date('fechaid', 'yyyyMMdd'))

# Extraer año y mes de la columna 'fechaid'
df_filtered = df_filtered.withColumn('Año', F.year('fechaid'))
df_filtered = df_filtered.withColumn('Mes', F.month('fechaid'))

# Agrupar por año, mes y codigoprocedimiento
result_df = df_filtered.groupBy('Año', 'Mes', 'codigoprocedimiento').agg(
    F.countDistinct('personaid').alias('Número_de_Personas'),
    F.sum('personaid_count').alias('Número_de_Atenciones')
)

# Mostrar los resultados
result_df.show(100)

# Imprimir los nombres de las columnas agrupadas para verificar
print("Nombres de las columnas agrupadas:")
print(result_df.columns)

+----+---+-------------------+------------------+--------------------+
| Año|Mes|codigoprocedimiento|Número_de_Personas|Número_de_Atenciones|
+----+---+-------------------+------------------+--------------------+
|2009|  7|             890409|               826|                8449|
|2009|  5|             902204|               542|                8862|
|2009|  4|             241101|                 2|                  23|
|2009| 11|             881502|                61|                 313|
|2009| 11|             901102|                29|                 414|
|2009|  7|             954107|               725|                5892|
|2009|  5|             990213|                25|                 174|
|2009|  7|             882101|                 8|                 518|
|2009|  5|             939400|              1140|               22414|
|2009|  9|             873431|               672|                4786|
|2009|  7|             452301|               354|                5455|
|2009|

In [395]:
#se escribe el parquet tabla 7
result_df.write.parquet("hdfs:/analyticsdata/7.Tabla_anio_mes_codigoprocedimiento_2009.parquet") 

# 8. Agrupado por año, mes, municipio y sexo

In [396]:
# Seleccionar solo las columnas deseadas
columns_to_keep = [
    'personaid', 'codigoprocedimiento', 'tipoeventoripsdesc', 'fechaid',
    'sexodesc', 'tipodiagnosticoprincipalcd', 'edad', 'dxegreso', 'dxprincipal', 'municipiocd', 'personaid_count'
]
df_filtered = df2.select(*columns_to_keep)

# Convertir 'fechaid' a tipo fecha si es necesario
df_filtered = df_filtered.withColumn('fechaid', F.to_date('fechaid', 'yyyyMMdd'))

# Extraer año y mes de la columna 'fechaid'
df_filtered = df_filtered.withColumn('Año', F.year('fechaid'))
df_filtered = df_filtered.withColumn('Mes', F.month('fechaid'))

# Agrupar por año, mes y codigoprocedimiento
result_df = df_filtered.groupBy('Año', 'Mes', 'municipiocd','sexodesc').agg(
    F.countDistinct('personaid').alias('Número_de_Personas'),
    F.sum('personaid_count').alias('Número_de_Atenciones')
)

# Mostrar los resultados
result_df.show(100)

# Imprimir los nombres de las columnas agrupadas para verificar
print("Nombres de las columnas agrupadas:")
print(result_df.columns)

+----+---+-----------+---------+------------------+--------------------+
| Año|Mes|municipiocd| sexodesc|Número_de_Personas|Número_de_Atenciones|
+----+---+-----------+---------+------------------+--------------------+
|2009|  6|      68001| FEMENINO|             14904|              187530|
|2009|  3|      15476|MASCULINO|               261|                1375|
|2009| 11|      47058| FEMENINO|               984|                7702|
|2009|  7|      15087|MASCULINO|                83|                 306|
|2009| 10|      52019| FEMENINO|               259|                4642|
|2009|  9|      73055| FEMENINO|               144|                1736|
|2009| 12|      25200|MASCULINO|               112|                 776|
|2009|  8|      86001| FEMENINO|               462|                3857|
|2009|  4|      25839|MASCULINO|               324|                1648|
|2009|  8|      15632| FEMENINO|               371|                4989|
|2009| 12|      17380|MASCULINO|              1434|

In [397]:
#se escribe el parquet tabla 8
result_df.write.parquet("hdfs:/analyticsdata/8.Tabla_anio_mes_municipio_sexo_2009.parquet") 

# 9. Agrupado por año, mes, tipoeventorips y sexo

In [398]:
# Seleccionar solo las columnas deseadas
columns_to_keep = [
    'personaid', 'codigoprocedimiento', 'tipoeventoripsdesc', 'fechaid',
    'sexodesc', 'tipodiagnosticoprincipalcd', 'edad', 'dxegreso', 'dxprincipal', 'municipiocd', 'personaid_count'
]
df_filtered = df2.select(*columns_to_keep)

# Convertir 'fechaid' a tipo fecha si es necesario
df_filtered = df_filtered.withColumn('fechaid', F.to_date('fechaid', 'yyyyMMdd'))

# Extraer año y mes de la columna 'fechaid'
df_filtered = df_filtered.withColumn('Año', F.year('fechaid'))
df_filtered = df_filtered.withColumn('Mes', F.month('fechaid'))

# Agrupar por año, mes y codigoprocedimiento
result_df = df_filtered.groupBy('Año', 'Mes', 'tipoeventoripsdesc','sexodesc').agg(
    F.countDistinct('personaid').alias('Número_de_Personas'),
    F.sum('personaid_count').alias('Número_de_Atenciones')
)

# Mostrar los resultados
result_df.show(100)

# Imprimir los nombres de las columnas agrupadas para verificar
print("Nombres de las columnas agrupadas:")
print(result_df.columns)

+----+----+--------------------+-----------+------------------+--------------------+
| Año| Mes|  tipoeventoripsdesc|   sexodesc|Número_de_Personas|Número_de_Atenciones|
+----+----+--------------------+-----------+------------------+--------------------+
|2009|   1|PROCEDIMIENTOS DE...|   FEMENINO|            255549|             2734138|
|2009|   4|PROCEDIMIENTOS DE...|   FEMENINO|            224726|             2645121|
|2009|   3|           CONSULTAS|   FEMENINO|            609038|             5284901|
|2009|   6|   HOSPITALIZACIONES|   FEMENINO|             10304|              200734|
|2009|  11|           CONSULTAS|   FEMENINO|            461845|             4064412|
|2009|  10|   HOSPITALIZACIONES|  MASCULINO|              9804|              175150|
|2009|   9|PROCEDIMIENTOS DE...|   FEMENINO|            255516|             3067780|
|2009|   9|           URGENCIAS|   FEMENINO|             25062|              261357|
|2009|   6|           CONSULTAS|       null|              8229|  

In [399]:
#se escribe el parquet tabla 9
result_df.write.parquet("hdfs:/analyticsdata/9.Tabla_anio_mes_tipoeventoripsdesc_sexo_2009.parquet") 

se comprueba que el archivo quedo guardado correctamente al leerlo 

In [400]:
# Leer el archivo .parquet y se guarda en un DataFrame
df_leer = spark.read.parquet("hdfs:/analyticsdata/9.Tabla_anio_mes_tipoeventoripsdesc_sexo_2009.parquet")

# Mostrar el contenido del DataFrame
df_leer.show()

+----+---+--------------------+---------+------------------+--------------------+
| Año|Mes|  tipoeventoripsdesc| sexodesc|Número_de_Personas|Número_de_Atenciones|
+----+---+--------------------+---------+------------------+--------------------+
|2009|  2|           URGENCIAS| FEMENINO|             15554|              172547|
|2009|  8|           URGENCIAS| FEMENINO|             26694|              279334|
|2009|  5|           URGENCIAS|     null|               572|                3990|
|2009|  8|           CONSULTAS|     null|              8542|               53237|
|2009|  5|   HOSPITALIZACIONES|MASCULINO|              6336|              112052|
|2009|  7|PROCEDIMIENTOS DE...|     null|              2849|               25528|
|2009|  6|           CONSULTAS|MASCULINO|            404998|             2686651|
|2009|  9|           CONSULTAS|MASCULINO|            453228|             3028436|
|2009|  2|           URGENCIAS|MASCULINO|             14054|              109938|
|2009|  5|      

# 10. Agrupado por año, mes, edad

In [159]:
# Seleccionar solo las columnas deseadas
columns_to_keep = [
    'personaid', 'codigoprocedimiento', 'tipoeventoripsdesc', 'fechaid',
    'sexodesc', 'tipodiagnosticoprincipalcd', 'edad', 'dxegreso', 'dxprincipal', 'municipiocd', 'personaid_count'
]
df_filtered = df2.select(*columns_to_keep)

# Convertir 'fechaid' a tipo fecha si es necesario
df_filtered = df_filtered.withColumn('fechaid', F.to_date('fechaid', 'yyyyMMdd'))

# Extraer año y mes de la columna 'fechaid'
df_filtered = df_filtered.withColumn('Año', F.year('fechaid'))
df_filtered = df_filtered.withColumn('Mes', F.month('fechaid'))

# Agrupar por año, mes y codigoprocedimiento
result_df = df_filtered.groupBy('Año', 'Mes', 'edad').agg(
    F.countDistinct('personaid').alias('Número_de_Personas'),
    F.sum('personaid_count').alias('Número_de_Atenciones')
)

# Mostrar los resultados
result_df.show(100)

# Imprimir los nombres de las columnas agrupadas para verificar
print("Nombres de las columnas agrupadas:")
print(result_df.columns)

+----+---+----+------------------+--------------------+
| Año|Mes|edad|Número_de_Personas|Número_de_Atenciones|
+----+---+----+------------------+--------------------+
|2009| 12|  70|              6501|              122350|
|2009|  5|  45|             17622|              179847|
|2009| 11|  17|             23800|              159102|
|2009|  3|   7|             32862|              160898|
|2009|  9|  15|             27475|              171311|
|2009|  6|  24|             23869|              213453|
|2009|  9|  35|             21144|              179983|
|2009|  1|  31|             24376|              195595|
|2009| 11|  49|             13960|              134344|
|2009| 11| 102|                15|                 197|
|2009|  6|  62|              7645|              110101|
|2009|  6|  83|              2020|               40296|
|2009|  2|  28|             26491|              226861|
|2009|  4|  20|             21433|              191275|
|2009| 11|  71|              5535|              

In [161]:
#se escribe el parquet tabla 10
result_df.write.parquet("hdfs:/analyticsdata/10.Tabla_anio_mes_edad_2009.parquet") 

AnalysisException: [PATH_ALREADY_EXISTS] Path hdfs://localhost:9000/analyticsdata/10.Tabla_anio_mes_edad_2009.parquet already exists. Set mode as "overwrite" to overwrite the existing path.

# 11. Agrupado por año, mes, municipiocd, tipoeventoripsdesc, dxprincipal, sexodesc, edad

In [179]:
# Seleccionar solo las columnas deseadas
columns_to_keep = [
    'personaid', 'codigoprocedimiento', 'tipoeventoripsdesc', 'fechaid',
    'sexodesc', 'tipodiagnosticoprincipalcd', 'edad', 'dxegreso', 'dxprincipal', 'municipiocd', 'personaid_count'
]
df_filtered = df2.select(*columns_to_keep)

# Convertir 'fechaid' a tipo fecha si es necesario
df_filtered = df_filtered.withColumn('fechaid', F.to_date('fechaid', 'yyyyMMdd'))

# Extraer año y mes de la columna 'fechaid'
df_filtered = df_filtered.withColumn('Año', F.year('fechaid'))
df_filtered = df_filtered.withColumn('Mes', F.month('fechaid'))

# Agrupar por año, mes y codigoprocedimiento
result_df = df_filtered.groupBy('Año', 'Mes', 'municipiocd','tipoeventoripsdesc','dxprincipal','sexodesc','edad').agg(
    F.countDistinct('personaid').alias('Número_de_Personas'),
    F.sum('personaid_count').alias('Número_de_Atenciones')
)

# Mostrar los resultados
result_df.show(100)

# Imprimir los nombres de las columnas agrupadas para verificar
print("Nombres de las columnas agrupadas:")
print(result_df.columns)

+----+---+-----------+--------------------+-----------+---------+----+------------------+--------------------+
| Año|Mes|municipiocd|  tipoeventoripsdesc|dxprincipal| sexodesc|edad|Número_de_Personas|Número_de_Atenciones|
+----+---+-----------+--------------------+-----------+---------+----+------------------+--------------------+
|2009| 10|      05088|PROCEDIMIENTOS DE...|       D649| FEMENINO|  12|                 1|                   1|
|2009| 10|      73671|PROCEDIMIENTOS DE...|          1| FEMENINO|  48|                 2|                  18|
|2009|  1|      17380|PROCEDIMIENTOS DE...|       M255|MASCULINO|  66|                 1|                  17|
|2009|  7|      13836|           CONSULTAS|       N309| FEMENINO|  30|                 1|                   4|
|2009|  4|      25473|           CONSULTAS|       L989|MASCULINO|  25|                 1|                   1|
|2009|  7|      11001|           CONSULTAS|       Z000|MASCULINO|  41|                44|                 267|
|

In [180]:
#se escribe el parquet tabla 11
result_df.write.parquet("hdfs:/analyticsdata/11.Tabla_anio_mes_municipio_tipoeventorips_dxprincipal_sexo_edad_2009.parquet") 

# 12. Agrupado por año, mes, sexodesc, edad

In [181]:
# Seleccionar solo las columnas deseadas
columns_to_keep = [
    'personaid', 'codigoprocedimiento', 'tipoeventoripsdesc', 'fechaid',
    'sexodesc', 'tipodiagnosticoprincipalcd', 'edad', 'dxegreso', 'dxprincipal', 'municipiocd', 'personaid_count'
]
df_filtered = df2.select(*columns_to_keep)

# Convertir 'fechaid' a tipo fecha si es necesario
df_filtered = df_filtered.withColumn('fechaid', F.to_date('fechaid', 'yyyyMMdd'))

# Extraer año y mes de la columna 'fechaid'
df_filtered = df_filtered.withColumn('Año', F.year('fechaid'))
df_filtered = df_filtered.withColumn('Mes', F.month('fechaid'))

# Agrupar por año, mes y codigoprocedimiento
result_df = df_filtered.groupBy('Año', 'Mes','sexodesc', 'edad').agg(
    F.countDistinct('personaid').alias('Número_de_Personas'),
    F.sum('personaid_count').alias('Número_de_Atenciones')
)

# Mostrar los resultados
result_df.show(100)

# Imprimir los nombres de las columnas agrupadas para verificar
print("Nombres de las columnas agrupadas:")
print(result_df.columns)

+----+---+---------+----+------------------+--------------------+
| Año|Mes| sexodesc|edad|Número_de_Personas|Número_de_Atenciones|
+----+---+---------+----+------------------+--------------------+
|2009|  6|MASCULINO|  20|              7546|               42302|
|2009|  9|MASCULINO|  89|               328|                4951|
|2009|  8| FEMENINO|  88|               498|               59915|
|2009|  8|MASCULINO|  85|               582|               11166|
|2009| 12| FEMENINO|  95|               166|                2807|
|2009|  9|MASCULINO|  64|              3107|               42802|
|2009| 10|MASCULINO|  26|              8213|               44356|
|2009| 11| FEMENINO|  55|              6751|               84164|
|2009| 10| FEMENINO|  48|              8880|               91666|
|2009|  6|MASCULINO|   4|             14321|              103509|
|2009|  2| FEMENINO|  10|             12851|               71171|
|2009|  4| FEMENINO|  75|              2557|               55621|
|2009| 11|

In [182]:
#se escribe el parquet tabla 12
result_df.write.parquet("hdfs:/analyticsdata/12.Tabla_anio_mes_sexo_edad_2009.parquet") 

In [183]:
spark.stop()